### Instal BeutifulSoup

In [242]:
!pip install beautifulsoup4

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [243]:
import requests

### Get the data from the page

In [244]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [245]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#soup.prettify()

In [246]:
my_table = soup.find('table',{'class':'wikitable sortable'})
#my_table

### Let's extract the data from the table

In [247]:
#Generate lists
A=[]
B=[]
C=[]
D=[]

for row in my_table.findAll("tr"):
#    print('row: ', len(row))
    cells = row.findAll('td')
#    states=row.findAll('th') #To store second column data
#    print('cells: ', len(cells))
#    print('states: ',len(states))
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
#        B.append(states[0].find(text=True))
        C.append(cells[1].find(text=True))
        D.append(cells[2].find(text=True))

In [248]:
#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=C
df['Neighborhood']=D
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [249]:
df = df[df.Borough != 'Not assigned']

In [250]:
df = df.replace('\n','', regex=True)
df.shape

(212, 3)

### Let's remove the entries where Borough is listed as Not Assigned, also remove the new line symbol "\n"

In [251]:
import numpy as np
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

In [252]:
df.shape

(212, 3)

In [253]:
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

In [254]:
df.shape

(103, 3)

In [255]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data

--2018-11-26 16:55:10--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2018-11-26 16:55:11--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-11-26 16:55:14--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-11-26 16:55:14--  https://ibm.ent

In [256]:
geo_df = pd.read_csv('Geospatial_data.csv')

In [257]:
geo_df.index
geo_df.shape

(103, 3)

In [258]:
toronto_df = pd.merge(df, geo_df,  how='inner', left_on=['Postcode'], right_on = ['Postal Code'])

In [259]:
toronto_df.drop(columns=['Postal Code'], inplace=True)

In [260]:
toronto_df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Let's import the required Libraries

In [261]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


### Get the Coordinates of Toronto

In [262]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Plot the Toronto Neighborhoods on the Map

In [269]:
# create map of Toronto using latitude and longitude values
#Postcode	Borough	Neighbourhood	Latitude	Longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [270]:
# The code was removed by Watson Studio for sharing.

Your credentails:


### Let's Select the Boroughs which contain the name Toronto and plot them on the map or we can select all buroughs
However we have to remove a couple of Boroughs as the have no data in Foursquare, which causes a mis match in Vector size

In [271]:
#tcity_df = toronto_df[toronto_df['Borough'].str.contains("Toronto")]
#tcity_df = toronto_df 'Islington Avenue', 'Upper Rouge'
#tcity_df = df[~df.C.str.contains("XYZ")]
temp_df = toronto_df[toronto_df.Neighborhood != 'Islington Avenue']
tcity_df = temp_df[temp_df.Neighborhood != 'Upper Rouge']

tcity_df.shape

(101, 5)

In [272]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [273]:
# create map of Toronto using latitude and longitude values
#Postcode	Borough	Neighbourhood	Latitude	Longitude
map_tcity = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(tcity_df['Latitude'], tcity_df['Longitude'], tcity_df['Borough'], tcity_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tcity)  
    
map_tcity

In [274]:
# Define Limit and radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [275]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        print(url)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)       

In [276]:
# Get Venues
tcity_venues = getNearbyVenues(names=tcity_df['Neighborhood'],
                                   latitudes=tcity_df['Latitude'],
                                   longitudes=tcity_df['Longitude']
                                  )

Rouge, Malvern
https://api.foursquare.com/v2/venues/explore?&client_id=4VL5OMH4UPKKYZBOTJ4ZYRF0EJRE53MJQJR2XEHGOLMOKVBY&client_secret=TXTSMPMIVKJWD5M445Z53Q1IRD4A1R5GARCOBBCGIMAYVERI&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100
Highland Creek, Rouge Hill, Port Union
https://api.foursquare.com/v2/venues/explore?&client_id=4VL5OMH4UPKKYZBOTJ4ZYRF0EJRE53MJQJR2XEHGOLMOKVBY&client_secret=TXTSMPMIVKJWD5M445Z53Q1IRD4A1R5GARCOBBCGIMAYVERI&v=20180605&ll=43.7845351,-79.16049709999999&radius=500&limit=100
Guildwood, Morningside, West Hill
https://api.foursquare.com/v2/venues/explore?&client_id=4VL5OMH4UPKKYZBOTJ4ZYRF0EJRE53MJQJR2XEHGOLMOKVBY&client_secret=TXTSMPMIVKJWD5M445Z53Q1IRD4A1R5GARCOBBCGIMAYVERI&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100
Woburn
https://api.foursquare.com/v2/venues/explore?&client_id=4VL5OMH4UPKKYZBOTJ4ZYRF0EJRE53MJQJR2XEHGOLMOKVBY&client_secret=TXTSMPMIVKJWD5M445Z53Q1IRD4A1R5GARCOBBCGIMAYVERI&v=20180605&ll=43.7709921,-79.21691740

In [277]:
print(tcity_venues.shape)
tcity_venues.shape

(2258, 7)


(2258, 7)

In [278]:
tcity_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


In [279]:
print('There are {} uniques categories.'.format(len(tcity_venues['Venue Category'].unique())))

There are 281 uniques categories.


### Analyze the Data

In [280]:
# one hot encoding
tcity_onehot = pd.get_dummies(tcity_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tcity_onehot['Neighborhood'] = tcity_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tcity_onehot.columns[-1]] + list(tcity_onehot.columns[:-1])
tcity_onehot = tcity_onehot[fixed_columns]

tcity_onehot.shape

(2258, 281)

In [281]:
tcity_onehot.shape

(2258, 281)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [282]:
tcity_grouped = tcity_onehot.groupby('Neighborhood').mean().reset_index()
tcity_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar

In [283]:
print(tcity_grouped.Neighborhood.shape)
print(tcity_merged.Neighbourhood.shape)
#idx1 = pd.Index(tcity_grouped.Neighborhood)
#idx2 = pd.Index(tcity_merged.Neighbourhood)
#df[~tcity_grouped.Neighborhood.isin(tcity_merged.Neighbourhood)]
#idx2.difference(idx1).values

(101,)
(101,)


### Let's get Data for each othe the Neighborhoods

In [284]:
num_top_venues = 5

for hood in tcity_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tcity_grouped[tcity_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Gym  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Clothing Store  0.25
3             Breakfast Spot  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1                Coffee Shop  0.33
2                       Park  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.17
1           Pizza Place  0.08
2  Fast Food Restaurant  0.08
3              Pharmacy  0.08
4            Beer Store  0.08


----Alde

#### Let's put that into a *pandas* dataframe

In [285]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Select top 10 Venues from Each Neighborhood

In [292]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
print(columns)
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
print(neighborhoods_venues_sorted.shape)
print(tcity_grouped.shape)
neighborhoods_venues_sorted['Neighborhood'] = tcity_grouped['Neighborhood']
print(neighborhoods_venues_sorted.shape)

for ind in np.arange(tcity_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tcity_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']
(0, 11)
(101, 281)
(101, 11)


(101, 11)

### Cluster Neighborhoods
Tried Many Numbers but 5 looks like a better fit

In [294]:
# set number of clusters
kclusters = 5

tcity_grouped_clustering = tcity_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tcity_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0,
       0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 2], dtype=int32)

In [295]:
tcity_grouped_clustering.shape

(101, 280)

### Let's merge Clustering data with other data

In [296]:
tcity_merged = tcity_df

# add clustering labels
print(kmeans.labels_)
print(tcity_merged.shape)
print(kmeans.labels_.shape)
tcity_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tcity_merged = tcity_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tcity_merged.head() # check the last columns!

[0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 4 0 0 0 0 2 0 0 0 0 0 0 0 0 4 0 0 2 0 0 0 2 0 0 2 0 2 0 0 0 0 0 2
 0 2 0 1 0 0 2 2 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 2]
(101, 6)
(101,)


/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Design Studio
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,History Museum,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Electronics Store,Tech Startup,Pizza Place,Rental Car Location,Medical Center,Breakfast Spot,Mexican Restaurant,Women's Store,Discount Store,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Dog Run,Dim Sum Restaurant,Diner


### Map the Clustered Data

In [299]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tcity_merged['Latitude'], tcity_merged['Longitude'], tcity_merged['Neighborhood'], tcity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [300]:
tcity_merged.loc[tcity_merged['Cluster Labels'] == 0, tcity_merged.columns[[2] + list(range(6, tcity_merged.shape[1]))]]
#tcity_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Design Studio
1,"Highland Creek, Rouge Hill, Port Union",Bar,History Museum,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,Woburn,Coffee Shop,Korean Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
4,Cedarbrae,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Dog Run,Dim Sum Restaurant,Diner
5,Scarborough Village,Convenience Store,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
6,"East Birchmount Park, Ionview, Kennedy Park",Department Store,Train Station,Coffee Shop,Discount Store,Bus Station,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,"Clairlea, Golden Mile, Oakridge",Bakery,Park,Intersection,Ice Cream Shop,Bus Station,Bus Line,Metro Station,Soccer Field,Drugstore,Donut Shop
8,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant


In [302]:
tcity_merged.loc[tcity_merged['Cluster Labels'] == 1, tcity_merged.columns[[2] + list(range(6, tcity_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Performing Arts Venue,Café,Breakfast Spot,Yoga Studio,Bar,Grocery Store,Furniture / Home Store,Italian Restaurant,Falafel Restaurant


In [303]:
tcity_merged.loc[tcity_merged['Cluster Labels'] == 2, tcity_merged.columns[[2] + list(range(6, tcity_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",Electronics Store,Tech Startup,Pizza Place,Rental Car Location,Medical Center,Breakfast Spot,Mexican Restaurant,Women's Store,Discount Store,Dessert Shop
13,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Noodle House,Shopping Mall,Chinese Restaurant,Pharmacy,Drugstore,Donut Shop
17,Hillcrest Village,Mediterranean Restaurant,Golf Course,Dog Run,Pool,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
41,"The Danforth West, Riverdale",Greek Restaurant,Ice Cream Shop,Coffee Shop,Bookstore,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Liquor Store,Juice Bar,Restaurant
47,Davisville,Dessert Shop,Sandwich Place,Seafood Restaurant,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Restaurant,Pizza Place,Thai Restaurant
59,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Café,Aquarium,Pizza Place,Brewery,Sports Bar,Italian Restaurant,Scenic Lookout,Bar
63,Roselawn,Garden,Home Service,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
66,"Harbord, University of Toronto",Café,Bookstore,Restaurant,Japanese Restaurant,Coffee Shop,Bar,Bakery,French Restaurant,Sandwich Place,Comfort Food Restaurant
68,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boutique,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden
74,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store


In [304]:
tcity_merged.loc[tcity_merged['Cluster Labels'] == 3, tcity_merged.columns[[2] + list(range(6, tcity_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,"Flemingdon Park, Don Mills South",Coffee Shop,Asian Restaurant,Gym,Beer Store,General Entertainment,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Clothing Store


In [305]:
tcity_merged.loc[tcity_merged['Cluster Labels'] == 4, tcity_merged.columns[[2] + list(range(6, tcity_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,"The Beaches West, India Bazaar",Sandwich Place,Park,Movie Theater,Sushi Restaurant,Food & Drink Shop,Italian Restaurant,Ice Cream Shop,Fish & Chips Shop,Pub,Burrito Place
56,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Pub,Café,Cheese Shop,Farmers Market,Steakhouse,Seafood Restaurant
